## Imports modules required to run

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from openvino.inference_engine import IECore
from os import path, makedirs

sys.path.append("../utils")
from notebook_utils import download_file

## Settings

In [ ]:
class Files:
    def __init__(self, model: str, charlist_link: str, charlist_name: str = None):
        self.model = model
        self.charlist_link = charlist_link
        self.charlist_name = charlist_name if charlist_name is not None else self.charlist_link.split('/')[-1]

ie = IECore()

model_folder = "model"
data_folder = "data"
charlist_folder = f"{model_folder}/charlists"

precision = "FP16"

model_extensions = ("bin", "xml")

handwritten_japanese_model_name = "handwritten-japanese-recognition-0001"
handwritten_simplified_chinese_model_name = "handwritten-simplified-chinese-recognition-0001"

japaneses_charlist_link = "https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/kondate_nakayosi.txt"
simplified_chinese_charlist_link = "https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/scut_ept.txt"

japaneses_files = Files(handwritten_japanese_model_name, japaneses_charlist_link)
chinese_files = Files(handwritten_simplified_chinese_model_name, simplified_chinese_charlist_link)

## Download models and convert public model

If it is your first run models will download and convert here. It might take up to ten minutes. 

In [ ]:
makedirs(model_folder, exist_ok=True)

used_files = (chinese_files, japaneses_files)

for file_name in used_files:
    for extension in model_extensions:
        if not path.isfile(f'{model_folder}/intel/{file_name.model}/{precision}/{file_name.model}.{extension}'):
            download_command = f'omz_downloader --name {file_name.model} --output_dir {model_folder} --precision {precision}'
            ! $download_command
            
for file_name in used_files:
    if not path.isfile(f'{charlist_folder}/{file_name.charlist_name}'):
        download_file(file_name.charlist_link, directory=charlist_folder, show_progress=False)    

## Load the network

In [ ]:
currently_used_model = handwritten_simplified_chinese_model_name

net = ie.read_network(
    model=f"{model_folder}/intel/{currently_used_model}/{precision}/{currently_used_model}.xml"
)

exec_net = ie.load_network(net, "CPU")

recognition_output_layer = next(iter(exec_net.outputs))
recognition_input_layer = next(iter(exec_net.input_info))

## Load an Image

In [ ]:
# Create data folder
makedirs(data_folder, exist_ok=True)

# Download image
image_link = 'https://github.com/openvinotoolkit/open_model_zoo/raw/master/demos/handwritten_text_recognition_demo/python/data/handwritten_simplified_chinese_test.png'
file_name = image_link.split('/')[-1]
download_file(image_link, directory=data_folder, show_progress=False)

# Text detection models expects image in BGR format
image = cv2.imread(f"{data_folder}/{file_name}", cv2.IMREAD_GRAYSCALE)
image_height, image_width = image.shape
aspect_ratio = image_width/image_height

# B,C,H,W = batch size, number of channels, height, width
_, _, H, W = net.input_info[recognition_input_layer].input_data.shape

# Resize image to meet network expected input sizes
resized_image = cv2.resize(image, (int(H*aspect_ratio), H), interpolation=cv2.INTER_AREA)
resized_image = np.pad(resized_image, ((0, 0), (0, W - int(H*aspect_ratio))), mode='edge')

# Reshape to network input shape
input_image = resized_image[None, None, :, :]

plt.imshow(resized_image);

In [ ]:
# Get dictionary to encode output, based on model documentation
used_charlist = chinese_files.charlist_name if currently_used_model == handwritten_simplified_chinese_model_name else japaneses_files.charlist_name

blank_char = '~'

with open(f"{charlist_folder}/{used_charlist}", 'r', encoding='utf-8') as charlist:
    letters = blank_char + ''.join(line.strip() for line in charlist)

In [ ]:
predictions = exec_net.infer(inputs={recognition_input_layer: input_image})[recognition_output_layer]
predictions = np.squeeze(predictions)
predictions_c = np.argsort(predictions, axis=1)[:,-2:][:,::-1]

output_text = []
cnt, prob = 0, 0
for i, p in enumerate(predictions_c):
    if p[0] != 0:
        cnt += 1
        if 0 in p:
            prob += predictions[i][p[0]] - predictions[i][0]

for i, p in enumerate(predictions_c):
    if p[0] != 0:
        if (0 in p and predictions[i][p[0]] - predictions[i][0] > prob/cnt) or 0 not in p:
            output_text.append(letters[p[0]])

print(''.join(output_text))